See https://github.com/cmoscardi/embedded_d3_example/blob/master/Embedded_D3.ipynb

In [6]:
%%javascript
require.config({
  paths: {
    d3: "//d3js.org/d3.v4"
  }
});

<IPython.core.display.Javascript object>

In [7]:
%%javascript
element.append("<div id='chart1'></div>");

<IPython.core.display.Javascript object>

In [8]:
%%html
<style>

		.link {
			stroke: #000;
			stroke-width: 1.5px;
		}

		.node {
			fill: #000;
			stroke: #fff;
			stroke-width: 1.5px;
		}


		.node.a { fill: #1f77b4; }
		.node.b { fill: #ff7f0e; }
		.node.c { fill: #2ca02c; }


	</style>

In [9]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  
  //set data
    var width = 960,
			height = 500;

			var color = d3.scaleOrdinal(d3.schemeCategory10);

			var nodes = [],
			links = [];



			var simulation = d3.forceSimulation()
			.force("link", d3.forceLink().distance(200).strength(.6))
			.force("charge", d3.forceManyBody())
			// use forceX and forceY instead to change the relative positioning
			// .force("centering", d3.forceCenter(width/2, height/2))
			.force("x", d3.forceX(width/2))
			.force("y", d3.forceY(height/2))
			.on("tick", tick);



			var a = createNode("a");
			var b = createNode("b");
			var c = createNode("c");

			nodes= [a];
			start();

			// 1. add the links
			setTimeout(function() {
				nodes= [a,b,c];
				links.push({source: a, target: b}, {source: a, target: c}, {source: b, target: c});
				start();
			}, 1000);

			// 2. Remove node B and associated links.
			setTimeout(function() {
				// remove b
				nodes = [a,c];
				// remove a-b
				links = [links[1]];
				start();
			}, 2000);


		// Add node B back.
		setTimeout(function() {
			b = createNode("b")
			nodes.push(b);
			links.push({source: a, target: b}, {source: b, target: c});
			start();
		}, 3000);

		function createNode(id) {
			return {id: id, x: width/2, y:height/2}
		}

		function start() {
			
			var nodeElements = svg.selectAll(".node").data(nodes, function(d){return d.id});
			var linkElements = svg.selectAll(".link").data(links);

			nodeElements.enter().append("circle").attr("class", function(d) {return "node " + d.id; }).attr("r", 8);
			linkElements.enter().insert("line", ".node").attr("class", "link");

			nodeElements.exit().remove();
			linkElements.exit().remove();

			simulation.nodes(nodes)
			simulation.force("link").links(links)
			simulation.restart();
		}

		function tick() {
			var nodeElements = svg.selectAll(".node");
			var linkElements = svg.selectAll(".link");

			nodeElements.attr("cx", function(d,i) {return d.x; })
			.attr("cy", function(d) { return d.y; })

			linkElements.attr("x1", function(d) { return d.source.x; })
			.attr("y1", function(d) { return d.source.y; })
			.attr("x2", function(d) { return d.target.x; })
			.attr("y2", function(d) { return d.target.y; });


		}
    
});

<IPython.core.display.Javascript object>

In [11]:
%%html
<style>

.links line {
  stroke: #999;
  stroke-opacity: 0.6;
}

.nodes circle {
  stroke: #fff;
  stroke-width: 1.5px;
}
	</style>

In [13]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  var  width = 960,
    height = 600;
  //set data
    var color = d3.scaleOrdinal(d3.schemeCategory20);

var simulation = d3.forceSimulation()
    .force("link", d3.forceLink().id(function(d) { return d.id; }))
    .force("charge", d3.forceManyBody())
    .force("center", d3.forceCenter(width / 2, height / 2));

d3.json("miserables.json", function(error, graph) {
  if (error) throw error;

  var link = svg.append("g")
      .attr("class", "links")
    .selectAll("line")
    .data(graph.links)
    .enter().append("line")
      .attr("stroke-width", function(d) { return Math.sqrt(d.value); });

  var node = svg.append("g")
      .attr("class", "nodes")
    .selectAll("circle")
    .data(graph.nodes)
    .enter().append("circle")
      .attr("r", 5)
      .attr("fill", function(d) { return color(d.group); })
      .call(d3.drag()
          .on("start", dragstarted)
          .on("drag", dragged)
          .on("end", dragended));

  node.append("title")
      .text(function(d) { return d.id; });

  simulation
      .nodes(graph.nodes)
      .on("tick", ticked);

  simulation.force("link")
      .links(graph.links);

  function ticked() {
    link
        .attr("x1", function(d) { return d.source.x; })
        .attr("y1", function(d) { return d.source.y; })
        .attr("x2", function(d) { return d.target.x; })
        .attr("y2", function(d) { return d.target.y; });

    node
        .attr("cx", function(d) { return d.x; })
        .attr("cy", function(d) { return d.y; });
  }
});

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x;
  d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x;
  d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null;
  d.fy = null;
}
    
});

<IPython.core.display.Javascript object>

In [14]:
%%html
<style>

.node {
  stroke: #fff;
  stroke-width: 1.5px;
}

.link {
  fill: none;
  stroke: #bbb;
}
</style>

In [15]:
%%javascript
require(['d3'], function(d3){
  //a weird idempotency thing
  $("#chart1").remove();
  //create canvas
  element.append("<div id='chart1'></div>");
  $("#chart1").width("960px");
  $("#chart1").height("600px");        
  var margin = {top: 20, right: 20, bottom: 30, left: 40};
  var width = 880 - margin.left - margin.right;
  var height = 500 - margin.top - margin.bottom;
  var svg = d3.select("#chart1").append("svg")
    .style("position", "relative")
    .style("max-width", "960px")
    .attr("width", width + "px")
    .attr("height", (height + 50) + "px")
    .append("g")
    .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
  var  width = 960,
    height = 600;
  //set data
    var color = d3.scaleOrdinal(d3.schemeCategory20);

var simulation = d3.forceSimulation()
    .force("link", d3.forceLink().distance(10).strength(0.5))
    .force("charge", d3.forceManyBody())
    .force("center", d3.forceCenter(width / 2, height / 2));

d3.json("miserables.json", function(error, graph) {
  if (error) throw error;

  var nodes = graph.nodes,
      nodeById = d3.map(nodes, function(d) { return d.id; }),
      links = graph.links,
      bilinks = [];

  links.forEach(function(link) {
    var s = link.source = nodeById.get(link.source),
        t = link.target = nodeById.get(link.target),
        i = {}; // intermediate node
    nodes.push(i);
    links.push({source: s, target: i}, {source: i, target: t});
    bilinks.push([s, i, t]);
  });

  var link = svg.selectAll(".link")
    .data(bilinks)
    .enter().append("path")
      .attr("class", "link");

  var node = svg.selectAll(".node")
    .data(nodes.filter(function(d) { return d.id; }))
    .enter().append("circle")
      .attr("class", "node")
      .attr("r", 5)
      .attr("fill", function(d) { return color(d.group); })
      .call(d3.drag()
          .on("start", dragstarted)
          .on("drag", dragged)
          .on("end", dragended));

  node.append("title")
      .text(function(d) { return d.id; });

  simulation
      .nodes(nodes)
      .on("tick", ticked);

  simulation.force("link")
      .links(links);

  function ticked() {
    link.attr("d", positionLink);
    node.attr("transform", positionNode);
  }
});

function positionLink(d) {
  return "M" + d[0].x + "," + d[0].y
       + "S" + d[1].x + "," + d[1].y
       + " " + d[2].x + "," + d[2].y;
}

function positionNode(d) {
  return "translate(" + d.x + "," + d.y + ")";
}

function dragstarted(d) {
  if (!d3.event.active) simulation.alphaTarget(0.3).restart();
  d.fx = d.x, d.fy = d.y;
}

function dragged(d) {
  d.fx = d3.event.x, d.fy = d3.event.y;
}

function dragended(d) {
  if (!d3.event.active) simulation.alphaTarget(0);
  d.fx = null, d.fy = null;
}
    
});

<IPython.core.display.Javascript object>